In [1]:
import config
from feature_generator import FeatureGenerator

import keras
import pickle
from datetime import datetime
from model import net
import os 
accuracy_values = list()
accuracy_sum = 0.0
accuracy_value = 0.0
precision_values = list()
precision_sum = 0.0
precision_value = 0.0
recall_values = list()
recall_sum = 0.0
recall_value = 0.0
f1_values = list()
f1_sum = 0.0
f1_value = 0.0
training_time_seconds = list()

args = ""


/home/botgod/anaconda3/envs/sem6clone/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

def load_model(checkpoint_dir, model_name):#, custom_objects):
    """load prediction model

    Keyword arguments:
    checkpoint_dir -- directory path
    model_name -- decide which model to load
    """
    model = net()
    model.load(checkpoint_dir, model_name)#, custom_objects)
    return model


# Test on BPIC17

In [3]:


loss = 'mae'
regression = True

#dependencies = {'mse': 'mse', 'mae': 'mae'}

# test the model*(real)
checkpoint_dir = './checkpoints/'
#modelname_next_act = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_activity'
modelname_next_time = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_timestamp'

# load prediction model
#model_next_act = load_model(checkpoint_dir, modelname_next_act)
model_next_time = load_model(checkpoint_dir, modelname_next_time)#, custom_objects=dependencies)
contextual_info = True


# (CHANGED)
est_dir = './estimation/'
estname_next_time = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_timestamp'
# load estimation model
est_next_time = load_model(est_dir, estname_next_time)#, custom_objects=dependencies)


In [4]:

# load the data 

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

filename = os.path.join(project_root, "sample_data/real_Dilyar/", "modi_assignment_filtered_test.csv" ) 
model_name = "modi_assignment_filtered_test.csv" + "next_timestamp"

# load data
FG = FeatureGenerator()
df = FG.create_initial_log(filename)

test_df = df
test_df = FG.order_csv_time(test_df)
test_df = FG.queue_level(test_df)
test_df.to_csv('./test_data2.csv')
state_list = FG.get_states(test_df)
test_X, test_Y_Event, test_Y_Time = FG.one_hot_encode_history(test_df, "./estimation/"+"modi_BPI_2012_dropna_filter_act.csv")
print('test_X shape:', test_X)
print('test_Y_Time shape:', test_Y_Time.shape)
print('test_Y_Event shape:', test_Y_Event.shape)


act_char_to_int: {'W-Afhandelen leads-START': 0, 'W-Beoordelen fraude-START': 1, 'W-Completeren aanvraag-START': 2, 'W-Nabellen incomplete dossiers-START': 3, 'W-Nabellen offertes-START': 4, 'W-Valideren aanvraag-START': 5, '!': 6}
act_int_to_char: {0: 'W-Afhandelen leads-START', 1: 'W-Beoordelen fraude-START', 2: 'W-Completeren aanvraag-START', 3: 'W-Nabellen incomplete dossiers-START', 4: 'W-Nabellen offertes-START', 5: 'W-Valideren aanvraag-START', 6: '!'}
res_char_to_int: {'10138': 0, '10188': 1, '10228': 2, '10609': 3, '10629': 4, '10789': 5, '10809': 6, '10861': 7, '10863': 8, '10881': 9, '10889': 10, '10899': 11, '10909': 12, '10910': 13, '10912': 14, '10913': 15, '10914': 16, '10929': 17, '10931': 18, '10932': 19, '10933': 20, '10935': 21, '10939': 22, '10972': 23, '10982': 24, '11000': 25, '11002': 26, '11003': 27, '11009': 28, '11019': 29, '11049': 30, '11119': 31, '11121': 32, '11122': 33, '11169': 34, '11179': 35, '11180': 36, '11181': 37, '11189': 38, '11201': 39, '11203':

#### Evaluation of RTM: 

In [5]:

if contextual_info:
    test_context_X = FG.generate_context_feature(test_df,state_list)
    print('train_context_X shape:', test_context_X.shape)
    if regression:
            
        scores = model_next_time.evaluate(test_X, test_context_X, test_Y_Time, loss, regression)
        print('Test loss:', scores)

train_context_X shape: (127532, 6)
127532/127532 [==============================] - 15s 120us/step
[765.7771437167598, 0.005622118370291378, 16993828.80222154, 765.7771437167598]
Test loss: [765.7771437167598, 0.005622118370291378, 16993828.80222154, 765.7771437167598]


### Evaluating the activity predictiom model:

In [6]:
loss = 'categorical_crossentropy'
regression = False
modelname_next_act = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_activity'
model_next_act = load_model(checkpoint_dir, modelname_next_act)


In [7]:
if contextual_info:
    test_context_X = FG.generate_context_feature(test_df,state_list)
    print('train_context_X shape:', test_context_X.shape)
    scores = model_next_act.evaluate(test_X, test_context_X, test_Y_Event, loss, regression)
    print('Test loss:', scores)

train_context_X shape: (127532, 6)
127532/127532 [==============================] - 16s 123us/step
[5.133560740240686, 0.15468274629159431, 5.133560740240686]
Test loss: [5.133560740240686, 0.15468274629159431, 5.133560740240686]


# Test on BPIC12


In [9]:


loss = 'mae'
regression = True

# test the model*(real)
checkpoint_dir = './checkpoints/'
#modelname_next_act = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_activity'
modelname_next_time = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_timestamp'

# load prediction model
#model_next_act = load_model(checkpoint_dir, modelname_next_act)
model_next_time = load_model(checkpoint_dir, modelname_next_time)
contextual_info = True


# (CHANGED)
est_dir = './estimation/'
estname_next_time = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_timestamp'
# load estimation model
est_next_time = load_model(est_dir, estname_next_time)

# load the data 

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

filename = os.path.join(project_root, "sample_data/real/", "modi_BPI_2012_dropna_filter_act.csv" ) 

# load data
FG = FeatureGenerator()
df = FG.create_initial_log(filename)

test_df = df
test_df = FG.order_csv_time(test_df)
test_df = FG.queue_level(test_df)
#test_df.to_csv('./test_data2.csv')
state_list = FG.get_states(test_df)
test_X, test_Y_Event, test_Y_Time = FG.one_hot_encode_history(test_df, checkpoint_dir+"modi_BPI_2012_dropna_filter_act.csv")
print('test_X shape:', test_X.shape)
print('test_Y_Time shape:', test_Y_Time.shape)
print('test_Y_Event shape:', test_Y_Event.shape)


act_char_to_int: {'W-Afhandelen leads-START': 0, 'W-Beoordelen fraude-START': 1, 'W-Completeren aanvraag-START': 2, 'W-Nabellen incomplete dossiers-START': 3, 'W-Nabellen offertes-START': 4, 'W-Valideren aanvraag-START': 5, '!': 6}
act_int_to_char: {0: 'W-Afhandelen leads-START', 1: 'W-Beoordelen fraude-START', 2: 'W-Completeren aanvraag-START', 3: 'W-Nabellen incomplete dossiers-START', 4: 'W-Nabellen offertes-START', 5: 'W-Valideren aanvraag-START', 6: '!'}
res_char_to_int: {'10138': 0, '10188': 1, '10228': 2, '10609': 3, '10629': 4, '10789': 5, '10809': 6, '10861': 7, '10863': 8, '10881': 9, '10889': 10, '10899': 11, '10909': 12, '10910': 13, '10912': 14, '10913': 15, '10914': 16, '10929': 17, '10931': 18, '10932': 19, '10933': 20, '10935': 21, '10939': 22, '10972': 23, '10982': 24, '11000': 25, '11002': 26, '11003': 27, '11009': 28, '11019': 29, '11049': 30, '11119': 31, '11121': 32, '11122': 33, '11169': 34, '11179': 35, '11180': 36, '11181': 37, '11189': 38, '11201': 39, '11203':

### Evaluation of RTM 

In [10]:
if contextual_info:
    test_context_X = FG.generate_context_feature(test_df,state_list)
    print('train_context_X shape:', test_context_X.shape)
    if regression:
            
        scores = model_next_time.evaluate(test_X, test_context_X, test_Y_Time, loss, regression)
        print('Test loss:', scores)

train_context_X shape: (52831, 6)
52831/52831 [==============================] - 7s 129us/step
[11.18829912068082, 0.1542276314991535, 13829.843707765609, 11.18829912068082]
Test loss: [11.18829912068082, 0.1542276314991535, 13829.843707765609, 11.18829912068082]


### Evaluation of activity prediction model

In [13]:
loss = 'categorical_crossentropy'
regression = False
modelname_next_act = 'modi_BPI_2012_dropna_filter_act.csv' + 'next_activity'
model_next_act = load_model(checkpoint_dir, modelname_next_act)

In [14]:
if contextual_info:
    test_context_X = FG.generate_context_feature(test_df,state_list)
    print('train_context_X shape:', test_context_X.shape)
    scores = model_next_act.evaluate(test_X, test_context_X, test_Y_Event, loss, regression)
    print('Test loss:', scores)

train_context_X shape: (52831, 6)
52831/52831 [==============================] - 7s 132us/step
[0.7272079182145895, 0.6984157028977567, 0.7272079182145895]
Test loss: [0.7272079182145895, 0.6984157028977567, 0.7272079182145895]
